#### Import Libraries

In [1]:
import numpy as np
import pandas as pd

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [2]:
intOutFileName = "data_int.txt"

x = range(1, 10)
intArray = [i for i in x]
with open(intOutFileName, 'a') as outfile:
    outfile.write(str(intArray))

# for Windows
!type data_int.txt
# OR
# !more data_int.txt

# for Ubuntu
# !cat data_int.txt

[1, 2, 3, 4, 5, 6, 7, 8, 9][1, 2, 3, 4, 5, 6, 7, 8, 9][1, 2, 3, 4, 5, 6, 7, 8, 9]


In [3]:
floatOutFileName = "data_float.txt"

x = range(1, 5)
n = range(1, 5)

randomTable = [[np.random.uniform(low= i, high= j) for j in x] for i in n]
with open(floatOutFileName, 'a') as outfile:
    outfile.write(str(randomTable))

# for Windows
!type data_float.txt
# OR
# !more data_float.txt

# for Ubuntu
# !cat data_float.txt

[[1.0, 1.8586098038500882, 2.619591008791901, 2.3092165060335486], [1.3329784874787105, 2.0, 2.86157317309036, 3.2251632780692274], [2.156005830457306, 2.221727647189385, 3.0, 3.505618817031398], [3.5024817908042145, 3.132366179037138, 3.644716604597993, 4.0]][[1.0, 1.2431205203432387, 2.6300078562181146, 3.0012406899367527], [1.983095648195437, 2.0, 2.4732799111383743, 2.3323782568391964], [1.7019180009732189, 2.4286008124220295, 3.0, 3.0105971799033884], [3.4727852584551764, 2.0262580265035384, 3.0943636085199557, 4.0]][[1.0, 1.9508450246442612, 2.3318154267263544, 2.394839963012762], [1.200368343006148, 2.0, 2.1466099905540497, 3.565715194302885], [1.060087724469736, 2.43902878095702, 3.0, 3.6359478072286673], [3.8258295921002445, 2.5946362885836907, 3.4545050204118404, 4.0]]


In [4]:
df = pd.read_fwf('data_float.txt')
df.to_csv('data_float.csv')

2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [5]:
import json

filteredData = []

data = json.load(open('user_data.json'))
# print(data)
for parsedData in data:
    if (parsedData['CreditCardType'] == 'American Express'):
        filteredData.append(parsedData)

df = pd.DataFrame(data=filteredData)
df.to_csv("filtered_Data.csv")

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [6]:
import pandas as pd

mcDf = pd.read_csv('mushrooms_categorized.csv')
mcDf.head(len(mcDf))

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


In [7]:
groupedMcDf = mcDf.groupby(['class']).mean()
groupedMcDf

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
class,,,,,,,,,,,,,,,,,,,,,
0,3.266160,1.615970,4.581749,0.653992,4.334601,0.954373,0.285171,0.068441,6.622624,0.615970,...,1.798479,6.098859,6.064639,0.0,1.931559,1.125475,3.007605,3.201521,3.283270,1.148289
1,3.436159,2.055158,4.421859,0.159346,3.940756,0.995403,0.028601,0.567926,2.863636,0.514811,...,1.394280,5.512768,5.504597,0.0,2.002043,1.009193,1.522983,4.021450,4.031665,1.895812


In [8]:
groupedMcDf.to_json ('mushrooms_categorized.json')

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [11]:
def mergeNumbers(line, index):
    return chr(int(parsNumbers(line, index), 2))

def parsNumbers(line, index):
    return line[index:index + 6]

def readCard(file, cards):
    for line in file:
        for index in range(0, len(line), 6):
            cards += mergeNumbers(line, index)
    print(cards)

with open('credit_card.dat', 'rb') as file:
    cards = ""
    readCard(file, cards)

7648 5673 3775 2271
3257 8247 3354 2266
2722 0001 4011 6652
0661 3063 3742 3150
0432 1608 1462 4742
5827 2027 8785 7303
5774 8528 2087 1117
8140 1210 6352 2845
5764 1133 7301 7100
6456 1737 4126 6726
1228 8631 7382 0000
7051 0160 5374 3166
0618 3587 1630 6376
1545 5454 7444 5636
6735 3116 3202 6834
7287 5011 1547 8413
7033 2607 3328 4200
2568 5244 1874 5024
1684 2253 7570 7118
0672 2576 0575 6631
6332 8353 8787 1340
1813 3361 1175 4211
2477 6450 8840 2368
5512 3505 2563 1326
3083 7882 0621 0025
4521 5148 8045 0334
7563 3654 8713 5787
8324 2664 0476 5561
0565 2504 7168 3510
5107 5507 1767 0738
2462 1821 2448 1443
2788 0638 6861 6554
5851 5873 5474 0547
0670 1004 4013 2655
5874 5506 3048 0806
2805 5401 8462 1260
5083 8406 6310 1862
1076 1445 3013 2266
8440 4804 4844 5277
4758 6141 0686 1387
7586 0675 0315 2568
2544 1258 7432 5165
3474 5023 4434 5626
1410 0270 0434 5086
7315 4446 1104 4215
0224 7742 8300 0266
0170 2700 3145 0640
2006 2437 8054 1600
8142 4055 1776 0026
3026 7380 1241 1084


5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

FileNotFoundError: No such file or directory: 'images/data_format.png'

FileNotFoundError: No such file or directory: 'images/data_format.png'

<IPython.core.display.Image object>

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?